In [3]:
!pip install ncps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00


In [14]:
import os
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import stats
from ncps.tf import LTC
from ncps.wirings import AutoNCP
from google.colab import drive
from scipy.stats import shapiro

# Load data and set up model
drive.mount('/content/drive')

path_save_spectrogram = '/content/drive/MyDrive/npy/'
X_in = np.load(os.path.join(path_save_spectrogram, 'noisy_voice_amp_db.npy'))
X_ou = np.load(os.path.join(path_save_spectrogram, 'voice_amp_db.npy'))

X_in = (X_in + 46) / 50
X_ou = (X_ou - 6) / 82

x_train, x_val, y_train, y_val = train_test_split(X_in, X_ou, test_size=0.10, random_state=42)

# Reshape to include the time dimension
x_train_reshaped = x_train.reshape((x_train.shape[0], 1, x_train.shape[1], x_train.shape[2], 1))
x_val_reshaped = x_val.reshape((x_val.shape[0], 1, x_val.shape[1], x_val.shape[2], 1))

y_train_reshaped = y_train.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], -1))
y_val_reshaped = y_val.reshape((y_val.shape[0], y_val.shape[1], y_val.shape[2], -1))

# Ensure the same number of samples in validation inputs and outputs
assert x_val_reshaped.shape[0] == y_val_reshaped.shape[0], "Mismatch in number of samples between input and output"

# Build best model (with correct hyperparameters)
def build_model():
    best_num_filters = 32
    best_dense_units = 32
    best_learning_rate = 0.001
    ncp = LTC(AutoNCP(32, output_size=8), return_sequences=True)

    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(None, 128, 128, 1)),
        TimeDistributed(Conv2D(best_num_filters, (5, 5), activation="relu")),
        TimeDistributed(MaxPool2D()),
        TimeDistributed(Conv2D(best_num_filters, (5, 5), activation="relu")),
        TimeDistributed(MaxPool2D()),
        TimeDistributed(Flatten()),
        TimeDistributed(Dense(best_dense_units, activation="relu")),
        ncp,
        TimeDistributed(Dense(1))
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(best_learning_rate),
        loss='mse',
        metrics=['mae']
    )

    return model

# Load best model and weights
best_model = build_model()
best_model.load_weights('best_model_rnn-ltc-f_weights.keras')

# Generate predictions for validation set
y_val_pred = best_model.predict(x_val_reshaped)

# Flatten predicted values
y_val_pred_flat = y_val_pred.flatten()

# Flatten actual target values
y_val_reshaped_flat = y_val_reshaped.flatten()

# Ensure the same number of elements before calculating residuals
assert y_val_pred_flat.shape[0] == y_val_reshaped_flat.shape[0], "Shape mismatch after flattening"

# Calculate residuals
residuals_flat = y_val_reshaped_flat - y_val_pred_flat

# Continue with the rest of the residual analysis...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2/2 [==============================] - 1s 122ms/step


AssertionError: Shape mismatch after flattening